In [ ]:
# Display complete results in JSON format (equivalent to original method_out.json)
import json

print("Complete results (formatted JSON):")
print(json.dumps(results, indent=2))

## Complete Results Output

Here's the complete results in JSON format (equivalent to the original script's output file):

## Experiment Customization

You can easily modify this notebook to explore different scenarios:

### 1. **Change Controller Parameters**
```python
controller = DKWController(
    epsilon_target=0.05,    # Stricter error threshold
    delta=0.01,            # Higher confidence level  
    min_samples=50,        # Fewer samples required
    hysteresis=0.02       # Less oscillation protection
)
```

### 2. **Modify Sample Data**
Add more examples with different difficulty levels to the `sample_data` list.

### 3. **Adjust Random Seed**
Change `np.random.seed(42)` to see different error patterns.

### Key Observations:
- The **baseline always chooses fission** (conservative approach)
- The **proposed method adapts** based on observed error rates
- The DKW bound provides **statistical guarantees** about decision quality
- **Hysteresis prevents** rapid switching between fusion/fission modes

In [ ]:
# Display detailed results
print("=== DETAILED RESULTS ===")
print("\nFirst 10 examples comparison:")
print("ID          | Difficulty | Error | Baseline | Proposed")
print("-" * 55)
for i in range(min(10, len(sample_data))):
    example = sample_data[i]
    baseline = results["baseline"][i]
    proposed = results["proposed"][i]
    print(f"{example['id']:11} | {example['difficulty']:10.2f} | {baseline['error']:5} | {baseline['decision']:8} | {proposed['decision']}")

# Calculate summary statistics
baseline_fission = sum(1 for r in results["baseline"] if r["decision"] == "fission")
proposed_fission = sum(1 for r in results["proposed"] if r["decision"] == "fission")
proposed_fusion = sum(1 for r in results["proposed"] if r["decision"] == "fusion")

total_errors = sum(1 for r in results["baseline"] if r["error"])
total_examples = len(results["baseline"])

print(f"\n=== SUMMARY STATISTICS ===")
print(f"Total examples: {total_examples}")
print(f"Total errors occurred: {total_errors} ({total_errors/total_examples*100:.1f}%)")
print(f"\nBaseline (always fission): {baseline_fission}/{total_examples} = {baseline_fission/total_examples*100:.1f}%")
print(f"Proposed method:")
print(f"  - Fission decisions: {proposed_fission}/{total_examples} = {proposed_fission/total_examples*100:.1f}%")  
print(f"  - Fusion decisions:  {proposed_fusion}/{total_examples} = {proposed_fusion/total_examples*100:.1f}%")

## Results Analysis

Let's examine the results and compare the adaptive DKW controller against the conservative baseline.

In [ ]:
# Run the experiment
results = run_experiment(sample_data)

print("Experiment completed!")
print(f"Processed {len(results['baseline'])} examples")
print(f"Baseline decisions: {len(results['baseline'])} examples")
print(f"Proposed method decisions: {len(results['proposed'])} examples")

## Run the Experiment

Let's execute the experiment and see how the DKW controller adapts its decisions based on the observed error rates.

In [ ]:
def run_experiment(data):
    """Run DKW controller experiment with inline data."""
    controller = DKWController()
    results = {"baseline": [], "proposed": []}

    # Set random seed for reproducible results
    np.random.seed(42)

    for example in data:
        # Simulate error occurrence based on difficulty
        error = np.random.random() < example["difficulty"]
        controller.add_observation(float(error))
        decision = controller.decide()

        results["proposed"].append({
            "id": example["id"],
            "decision": decision,
            "error": error,
        })
        results["baseline"].append({
            "id": example["id"],
            "decision": "fission",  # Always conservative
            "error": error,
        })

    return results

## Experiment Function

The modified experiment function now uses inline data instead of reading from external files. This makes the notebook completely self-contained.

In [ ]:
# Sample dataset - inline data to make notebook self-contained
# This replaces the external JSON file reading from the original script
sample_data = [
    {"id": "example_000", "difficulty": 0.05},
    {"id": "example_001", "difficulty": 0.08}, 
    {"id": "example_002", "difficulty": 0.15},
    {"id": "example_003", "difficulty": 0.12},
    {"id": "example_004", "difficulty": 0.03},
    {"id": "example_005", "difficulty": 0.18},
    {"id": "example_006", "difficulty": 0.09},
    {"id": "example_007", "difficulty": 0.22},
    {"id": "example_008", "difficulty": 0.06},
    {"id": "example_009", "difficulty": 0.14},
    {"id": "example_010", "difficulty": 0.11},
    {"id": "example_011", "difficulty": 0.07},
    {"id": "example_012", "difficulty": 0.16},
    {"id": "example_013", "difficulty": 0.04},
    {"id": "example_014", "difficulty": 0.19},
]

print(f"Created sample dataset with {len(sample_data)} examples")
print("Difficulty range:", f"{min(ex['difficulty'] for ex in sample_data):.2f} - {max(ex['difficulty'] for ex in sample_data):.2f}")

## Experiment Setup

The experiment simulates a scenario where we have examples with varying difficulty levels. For each example:

1. **Error simulation**: Errors occur probabilistically based on the example's difficulty
2. **Controller decision**: The DKW controller decides between fusion and fission modes  
3. **Baseline comparison**: Compare against a conservative baseline that always chooses fission

### Sample Data Structure
Each example contains:
- `id`: Unique identifier for the example
- `difficulty`: Probability of error occurrence (0.0 to 1.0)

In [ ]:
@dataclass
class DKWController:
    """DKW-guided fusion/fission controller."""
    epsilon_target: float = 0.10
    delta: float = 0.05
    min_samples: int = 100
    hysteresis: float = 0.05

    samples: list = field(default_factory=list)
    current_state: str = "fission"

    def dkw_epsilon(self, n: int) -> float:
        """Compute DKW epsilon for n samples."""
        if n < 2:
            return 1.0
        return np.sqrt(np.log(2 / self.delta) / (2 * n))

    def add_observation(self, error: float) -> None:
        """Add error observation for calibration."""
        self.samples.append(error)

    def decide(self) -> str:
        """Make fusion/fission decision with DKW guarantee."""
        n = len(self.samples)
        if n < self.min_samples:
            return self.current_state

        epsilon = self.dkw_epsilon(n)
        empirical_error = np.mean(self.samples[-self.min_samples:])
        error_upper_bound = empirical_error + epsilon

        if self.current_state == "fusion":
            if error_upper_bound > self.epsilon_target + self.hysteresis:
                self.current_state = "fission"
        else:
            if error_upper_bound < self.epsilon_target - self.hysteresis:
                self.current_state = "fusion"

        return self.current_state

## DKW Controller Class

The `DKWController` implements a statistical decision-making algorithm that:

1. **Collects error observations** over time
2. **Computes confidence bounds** using the DKW inequality  
3. **Makes adaptive decisions** to switch between fusion and fission modes
4. **Includes hysteresis** to prevent rapid oscillation between states

### Key Parameters:
- `epsilon_target`: Target error rate threshold (default: 0.10)
- `delta`: Confidence level parameter (default: 0.05)
- `min_samples`: Minimum samples before making decisions (default: 100)
- `hysteresis`: Buffer to prevent oscillation (default: 0.05)

In [ ]:
"""Import required libraries"""
import json
import numpy as np
from dataclasses import dataclass, field

# DKW Controller Implementation (experiment_001)

This notebook demonstrates a DKW-guided fusion/fission controller that makes adaptive decisions based on error observations with statistical guarantees.

## Overview
The DKW (Dvoretzky-Kiefer-Wolfowitz) inequality provides a confidence bound for empirical distributions. This implementation uses it to adaptively switch between "fusion" and "fission" modes based on observed error rates.